In [83]:
# biblioteki
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [98]:
# Ścieżki do plików CSV i folderu ze zdjęciami
metadata_train_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M-metadata/DF20M-train_metadata_PROD.csv'
metadata_test_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M-metadata/DF20M-public_test_metadata_PROD.csv'
images_folder_path = '/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M'

# Wczytanie metadanych
metadata_train = pd.read_csv(metadata_train_path)
metadata_test = pd.read_csv(metadata_test_path)

In [99]:
metadata_train.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,level2Gid,level2Name,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path
0,2862684394,2020-09-17T00:00:00,2020.0,9.0,17.0,DK,"Langesø, Morud",17215.0,Mycena crocata (Schrad.) P.Kumm.,Fungi,...,DNK.5.14_1,Nordfyns,2862684394-136762,dead wood (including bark),Uffe Nygaard,55.433810,10.200162,10.0,Unmanaged deciduous woodland,2862684394-136762.JPG
1,2238502117,2017-08-16T00:00:00,2017.0,8.0,16.0,DK,Virum,10057.0,Agaricus augustus Fr.,Fungi,...,DNK.1.28_1,Rudersdal,2238502117-320693,soil,Michael Hansen,55.800171,12.478571,5.0,garden,2238502117-320693.JPG
2,2818074328,2020-07-23T00:00:00,2020.0,7.0,23.0,DK,"Gribskov, Enghavehus",20027.0,Russula olivacea (Schaeff.) Fr.,Fungi,...,DNK.1.20_1,Hillerød,2818074328-59948,soil,Anna ploug,55.976322,12.293878,1.0,Mixed woodland (with coniferous and deciduous ...,2818074328-59948.JPG
3,2238560017,2018-09-09T00:00:00,2018.0,9.0,9.0,DK,Færgelunden,11069.0,Boletus edulis Bull.,Fungi,...,DNK.1.11_1,Frederikssund,2238560017-331396,soil,Henrik Dalgaard,55.840458,12.027228,15.0,Deciduous woodland,2238560017-331396.JPG
4,2237970493,2011-08-20T00:00:00,2011.0,8.0,20.0,DK,Tofte Skov,17278.0,Mycena pseudocorticola Kühner,Fungi,...,DNK.3.1_1,Aalborg,2237970493-5168,bark,David Boertmann,56.844840,10.192650,75.0,Forest bog,2237970493-5168.JPG


In [108]:
metadata_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3640 entries, 0 to 3639
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gbifID                3640 non-null   int64  
 1   eventDate             3639 non-null   object 
 2   year                  3639 non-null   float64
 3   month                 3639 non-null   float64
 4   day                   3639 non-null   float64
 5   countryCode           3640 non-null   object 
 6   locality              3636 non-null   object 
 7   taxonID               3640 non-null   float64
 8   scientificName        3640 non-null   object 
 9   kingdom               3640 non-null   object 
 10  phylum                3640 non-null   object 
 11  class                 3640 non-null   object 
 12  order                 3640 non-null   object 
 13  family                3640 non-null   object 
 14  genus                 3640 non-null   object 
 15  specificEpithet      

In [101]:
# Upewnij się, że kolumna 'image_path' jest w obu ramkach danych
metadata_train['full_image_path'] = metadata_train['image_path'].apply(lambda x: os.path.join(images_folder_path, x))
metadata_test['full_image_path'] = metadata_test['image_path'].apply(lambda x: os.path.join(images_folder_path, x))

In [102]:
metadata_train['full_image_path'][0]

'/Users/milenabiernacka/Desktop/studia/magisterka/DF/DF20M/2862684394-136762.JPG'

In [103]:
metadata_test.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,level2Name,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,full_image_path
0,2238504270,2017-08-20T00:00:00,2017.0,8.0,20.0,DK,Store Dyrehave,20020.0,Russula nigricans (Bull.) Fr.,Fungi,...,Hillerød,2238504270-171705,soil,Dan Ingemann Jensen,55.905585,12.362367,75.0,Deciduous woodland,2238504270-171705.JPG,/Users/milenabiernacka/Desktop/studia/magister...
1,2238511676,2017-09-17T00:00:00,2017.0,9.0,17.0,DK,Store Hareskov,17238.0,Mycena haematopus (Pers.) P.Kumm.,Fungi,...,Furesø,2238511676-246956,dead wood (including bark),Tarquin Netherway,55.769675,12.401108,15.0,Deciduous woodland,2238511676-246956.JPG,/Users/milenabiernacka/Desktop/studia/magister...
2,2883271536,2020-10-18T00:00:00,2020.0,10.0,18.0,DK,Lillerød,17289.0,Mycena rosea (Schumach.) Gramberg,Fungi,...,Allerød,2883271536-291057,soil,Dan Ingemann Jensen,55.878946,12.359130,40.0,Deciduous woodland,2883271536-291057.JPG,/Users/milenabiernacka/Desktop/studia/magister...
3,2446759186,2019-10-26T00:00:00,2019.0,10.0,26.0,DK,Ollerup,10079.0,Agaricus impudicus (Rea) Pilát,Fungi,...,Svendborg,2446759186-271681,soil,Dan Schou,55.068441,10.515358,50.0,Unmanaged deciduous woodland,2446759186-271681.JPG,/Users/milenabiernacka/Desktop/studia/magister...
4,2382321410,2019-08-30T00:00:00,2019.0,8.0,30.0,DK,"Dyrehave, Valdemars Slot (inkl Pederskov)",19939.0,Russula chloroides (Krombh.) Bres.,Fungi,...,Svendborg,2382321410-191132,soil,Kirsten Bjørnsson,55.013310,10.644585,75.0,Deciduous woodland,2382321410-191132.JPG,/Users/milenabiernacka/Desktop/studia/magister...


In [104]:
metadata_train['scientificName'].value_counts()

scientificName
Mycena galericulata (Scop.) Gray       1099
Clitocybe nebularis (Batsch) Quél.     1003
Amanita muscaria (L.) Lam., 1783        863
Boletus edulis Bull.                    811
Amanita rubescens (Pers.) Gray          713
                                       ... 
Agaricus bisporus (J.E.Lange) Pilat      33
Russula laeta Jul.Schäff.                32
Clitocybe diatreta (Fr.) P.Kumm.         32
Clitocybe nitrophila Bon                 29
Russula insignis Quél.                   28
Name: count, Length: 182, dtype: int64

In [33]:
from sklearn.preprocessing import LabelEncoder

# Zakodowanie etykiet
label_encoder = LabelEncoder()
metadata_train['class_encoded'] = label_encoder.fit_transform(metadata_train['scientificName'])
metadata_test['class_encoded'] = label_encoder.transform(metadata_test['scientificName'])  # Uwaga: używamy transform, nie fit_transform

# Teraz 'class_encoded' zawiera zakodowane etykiety


In [34]:
metadata_train.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,full_image_path,class_encoded
0,2862684394,2020-09-17T00:00:00,2020.0,9.0,17.0,DK,"Langesø, Morud",17215.0,Mycena crocata (Schrad.) P.Kumm.,Fungi,...,2862684394-136762,dead wood (including bark),Uffe Nygaard,55.433810,10.200162,10.0,Unmanaged deciduous woodland,2862684394-136762.JPG,/Users/milenabiernacka/Desktop/studia/magister...,70
1,2238502117,2017-08-16T00:00:00,2017.0,8.0,16.0,DK,Virum,10057.0,Agaricus augustus Fr.,Fungi,...,2238502117-320693,soil,Michael Hansen,55.800171,12.478571,5.0,garden,2238502117-320693.JPG,/Users/milenabiernacka/Desktop/studia/magister...,1
2,2818074328,2020-07-23T00:00:00,2020.0,7.0,23.0,DK,"Gribskov, Enghavehus",20027.0,Russula olivacea (Schaeff.) Fr.,Fungi,...,2818074328-59948,soil,Anna ploug,55.976322,12.293878,1.0,Mixed woodland (with coniferous and deciduous ...,2818074328-59948.JPG,/Users/milenabiernacka/Desktop/studia/magister...,151
3,2238560017,2018-09-09T00:00:00,2018.0,9.0,9.0,DK,Færgelunden,11069.0,Boletus edulis Bull.,Fungi,...,2238560017-331396,soil,Henrik Dalgaard,55.840458,12.027228,15.0,Deciduous woodland,2238560017-331396.JPG,/Users/milenabiernacka/Desktop/studia/magister...,40
4,2237970493,2011-08-20T00:00:00,2011.0,8.0,20.0,DK,Tofte Skov,17278.0,Mycena pseudocorticola Kühner,Fungi,...,2237970493-5168,bark,David Boertmann,56.844840,10.192650,75.0,Forest bog,2237970493-5168.JPG,/Users/milenabiernacka/Desktop/studia/magister...,89


In [105]:
# Definicja ImageDataGeneratorów
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [107]:
metadata_train.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,full_image_path,class_index,class_index_str
0,2862684394,2020-09-17T00:00:00,2020.0,9.0,17.0,DK,"Langesø, Morud",17215.0,Mycena crocata (Schrad.) P.Kumm.,Fungi,...,dead wood (including bark),Uffe Nygaard,55.433810,10.200162,10.0,Unmanaged deciduous woodland,2862684394-136762.JPG,/Users/milenabiernacka/Desktop/studia/magister...,70,70
1,2238502117,2017-08-16T00:00:00,2017.0,8.0,16.0,DK,Virum,10057.0,Agaricus augustus Fr.,Fungi,...,soil,Michael Hansen,55.800171,12.478571,5.0,garden,2238502117-320693.JPG,/Users/milenabiernacka/Desktop/studia/magister...,1,1
2,2818074328,2020-07-23T00:00:00,2020.0,7.0,23.0,DK,"Gribskov, Enghavehus",20027.0,Russula olivacea (Schaeff.) Fr.,Fungi,...,soil,Anna ploug,55.976322,12.293878,1.0,Mixed woodland (with coniferous and deciduous ...,2818074328-59948.JPG,/Users/milenabiernacka/Desktop/studia/magister...,151,151
3,2238560017,2018-09-09T00:00:00,2018.0,9.0,9.0,DK,Færgelunden,11069.0,Boletus edulis Bull.,Fungi,...,soil,Henrik Dalgaard,55.840458,12.027228,15.0,Deciduous woodland,2238560017-331396.JPG,/Users/milenabiernacka/Desktop/studia/magister...,40,40
4,2237970493,2011-08-20T00:00:00,2011.0,8.0,20.0,DK,Tofte Skov,17278.0,Mycena pseudocorticola Kühner,Fungi,...,bark,David Boertmann,56.844840,10.192650,75.0,Forest bog,2237970493-5168.JPG,/Users/milenabiernacka/Desktop/studia/magister...,89,89


In [111]:
metadata_train['class_index_str'] = metadata_train['class_index'].astype('string')
metadata_test['class_index_str'] = metadata_test['class_index'].astype('string')

In [112]:
metadata_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32753 entries, 0 to 32752
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gbifID                32753 non-null  int64  
 1   eventDate             32748 non-null  object 
 2   year                  32748 non-null  float64
 3   month                 32748 non-null  float64
 4   day                   32748 non-null  float64
 5   countryCode           32753 non-null  object 
 6   locality              32737 non-null  object 
 7   taxonID               32753 non-null  float64
 8   scientificName        32753 non-null  object 
 9   kingdom               32753 non-null  object 
 10  phylum                32753 non-null  object 
 11  class                 32753 non-null  object 
 12  order                 32753 non-null  object 
 13  family                32753 non-null  object 
 14  genus                 32753 non-null  object 
 15  specificEpithet    

In [116]:
# Utwórz teraz zmniejszone próbki danych
train_sample = metadata_train.sample(frac=0.1, random_state=42) 
test_sample = metadata_test.sample(frac=0.1, random_state=42) 

# Lista wszystkich unikalnych klas z oryginalnego DataFrame
all_classes = sorted(metadata_train['scientificName'].unique().tolist())

# Tworzenie generatorów z jawnie określoną listą klas
train_generator_sample = train_datagen.flow_from_dataframe(
    dataframe=train_sample,
    x_col='full_image_path',
    y_col='scientificName',
    target_size=(224, 224),
    batch_size=32,
    classes=all_classes,  # Zapewnij listę wszystkich klas
    class_mode='categorical')

test_generator_sample = test_datagen.flow_from_dataframe(
    dataframe=test_sample,
    x_col='full_image_path',
    y_col='scientificName',
    target_size=(224, 224),
    batch_size=32,
    classes=all_classes,  # Zapewnij listę wszystkich klas
    class_mode='categorical')


Found 3275 validated image filenames belonging to 182 classes.
Found 364 validated image filenames belonging to 182 classes.


In [117]:
# Wczytanie MobileNetV2 bez górnych warstw
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Dodanie górnych warstw
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)  # liczba neuronów odpowiada liczbie klas

# Zbudowanie całego modelu
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilacja modelu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Trening modelu
model.fit(train_generator_sample, validation_data=test_generator_sample, epochs=10, steps_per_epoch=len(train_generator_sample), validation_steps=len(test_generator_sample))

Epoch 1/10
103/103 [==============================] - 135s 1s/step - loss: 4.5952 - accuracy: 0.0867 - val_loss: 15.7188 - val_accuracy: 0.0604
Epoch 2/10
103/103 [==============================] - 136s 1s/step - loss: 3.7979 - accuracy: 0.1728 - val_loss: 11.5603 - val_accuracy: 0.0385
Epoch 3/10
103/103 [==============================] - 140s 1s/step - loss: 3.2783 - accuracy: 0.2437 - val_loss: 10.7539 - val_accuracy: 0.0275
Epoch 4/10
103/103 [==============================] - 143s 1s/step - loss: 2.8607 - accuracy: 0.3023 - val_loss: 15.0076 - val_accuracy: 0.0110
Epoch 5/10
103/103 [==============================] - 151s 1s/step - loss: 2.4917 - accuracy: 0.3573 - val_loss: 20.2482 - val_accuracy: 0.0330
Epoch 6/10
103/103 [==============================] - 149s 1s/step - loss: 2.1427 - accuracy: 0.4321 - val_loss: 18.1709 - val_accuracy: 0.0192
Epoch 7/10
103/103 [==============================] - 144s 1s/step - loss: 1.9511 - accuracy: 0.4641 - val_loss: 12.0305 - val_accuracy:

In [118]:
test_loss, test_accuracy = model.evaluate(test_generator_sample, steps=test_generator_sample.samples // test_generator_sample.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

11/11 [==============================] - 7s 623ms/step - loss: 13.9522 - accuracy: 0.0312
Test Loss: 13.95217514038086, Test Accuracy: 0.03125


In [119]:
model.evaluate(test_generator_sample)

12/12 [==============================] - 7s 515ms/step - loss: 13.9115 - accuracy: 0.0302


[13.911470413208008, 0.030219780281186104]